# data

> Functionality for efficiently accessing data in WEAVE fits files

In [ ]:
# |default_exp data

In [ ]:
# |export

import os
import sys
import time
from functools import partial, wraps
from glob import glob
from typing import Callable

import xarray as xr
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm

## Accessing FITS tables as xarray Datasets via cached netCDF files

The approach of `qagmire` is to access WEAVE data stored on disk in FITS files on disk. To analyse this large, multi-dimensional dataset we utilize [`xarray`](https://docs.xarray.dev), making use of its ability to use `dask` to perform computations in parallel in a memory efficient and scaleable manner. To make this work we need to write the data to disk in a suitable format: netCDF files, then load those files as an `xarray` `Dataset`.

In [ ]:
# |export


class ViaNetCDF:
    """Access FITS tables as xarray Datasets via cached netCDF files.

    For each FITS table or image we wish to read, we will write a `read_*(fn)` function
    which reads the table from the single provided filename `fn` and returns a `Dataset`.
    Decorating such a function with an instance of this class adapts the function to take
    a list of FITS filenames and return a `Dataset` that lazily loads data from a cache
    of netCDF files. The cache is stored in the `netcdf_store` folder defined when the
    instance is initialised.

    When the decorated function is initially run, it repeatedly calls `single_via_netcdf`
    to apply the original `read_*` function to each FITS filename and save each resulting
    `Dataset` as a netCDF file, then opens them together and returns a combined,
    distributed `Dataset`. If a previously converted file is found in the `netcdf_store`,
    then the original `read_*` function is skipped and the netCDF file loaded directly.
    This caching vastly increase the speed of subsequent calls.

    If a source FITS file is changed, the corresponding files in `netcdf_store` can simply
    be deleted and they will be recreated on the next call of the decorated `read_*`
    function.

    For example,
    ```
    via_netcdf = ViaNetCDF()

    @via_netcdf
    def read_class_spec(fn):
        ...
    ```
    """

    def __init__(
        self,
        netcdf_store: str | None = None,  # folder in which to store the netCDF files
        progress=True,  # display a progress bar
    ):
        """Create a decorator that can extend a `read_*` function to multiple files.

        If no `netcdf_store` is provided it first checks for a `NETCDF_STORE` environment
        variable and falls back to a folder called `netcdf_store` in the user's home folder.
        """
        if netcdf_store is not None:
            self.netcdf_store = netcdf_store
        else:
            default = os.path.join(os.environ.get("HOME"), "netcdf_store")
            self.netcdf_store = os.environ.get("NETCDF_STORE", default)

        if progress:
            self.progress = partial(
                tqdm, desc="Locating and converting where necessary", file=sys.stdout
            )
        else:
            self.progress = lambda x: x

    def single_via_netcdf(
        self,
        read_function: Callable[[str], xr.Dataset],  # function to read a FITS file
        fn: str,  # filename of FITS file to read
    ):
        """Transform a FITS file to netCDF using the given read function.

        Returns the path of a netCDF file stored in `nedcdf_store`, containing the `Dataset` resulting
        from calling `read_function` with the supplied FITS filename `fn`. If the netCDF file already
        exists, the filename is immediately returned.
        """
        fn_netcdf = os.path.join(
            self.netcdf_store, *os.path.normpath(fn).split(os.sep)[-3:]
        )
        table = read_function.__name__.replace("read_", "")
        fn_netcdf = os.path.splitext(fn_netcdf)[0]
        fn_netcdf = f"{fn_netcdf}_{table}.nc"
        if not os.path.exists(fn_netcdf):
            ds = read_function(fn)
            fn_base = os.path.splitext(os.path.basename(fn))[0]
            ds = ds.expand_dims({"filename": [fn_base]})
            os.makedirs(os.path.dirname(fn_netcdf), exist_ok=True)
            ds.to_netcdf(fn_netcdf)
        return fn_netcdf

    def __call__(
        self,
        read_function: Callable[[str], xr.Dataset],  # function to read a FITS file
    ):
        """Extend the functionality of `read_function` to multiple files.

        The wrapped `read_function` is adapted to take a list of FITS filenames and
        return a `Dataset` which lazily loads data from a cache of netCDF files.
        """

        @wraps(read_function)
        def netcdf_wrapper(fns):
            fns_netcdf = [
                self.single_via_netcdf(read_function, fn) for fn in self.progress(fns)
            ]
            print("Reading netCDF files... ", end="")
            start = time.perf_counter()
            data = xr.open_mfdataset(fns_netcdf, parallel=True)
            dt = time.perf_counter() - start
            print(f"took {dt:.2f} s.")
            return data

        return netcdf_wrapper

Now create a decorator to use below.

In [ ]:
# |exports

via_netcdf = ViaNetCDF()

## WEAVE data structures

### L2 stacks

Let's start by obtaining a list of L2 stack files and looking at an example.

In [ ]:
# |export


def _is_lowres(fn):
    """Check the header of FITS file `fn` to determine if it is low-resolution."""
    return "LR" in fits.getheader(fn)["RES-OBS"]


def get_weave_files(
    level="*",  # pattern to match to the file level, e.g. raw, L1, L2
    filetype="*",  # pattern to match to the file type, e.g. single, stack
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
    lowres=True,  # select low-res files, or high-res if False
):
    """Get a list of matching WEAVE files."""
    pattern = f"{level}/{date}/{filetype}_*{runid}*.fits"
    pattern = os.path.join(os.environ["WEAVEIO_ROOTDIR"], pattern)
    files = glob(pattern)
    files.sort()
    if lowres:
        files = [fn for fn in files if _is_lowres(fn)]
    else:
        files = [fn for fn in files if not _is_lowres(fn)]
    return files


def get_lr_l2_stack_files(
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
):
    return get_weave_files(
        level="L2", filetype="stack", date=date, runid=runid, lowres=True
    )

In [ ]:
lr_l2_stack_files = get_lr_l2_stack_files()
print(len(lr_l2_stack_files), "low-res L2 stack files")
fn = lr_l2_stack_files[0]
hdus = fits.open(fn)

17 low-res L2 stack files


WEAVE L2 stack files contain six extensions:

In [ ]:
print([hdu.name for hdu in hdus])

['PRIMARY', 'CLASS_TABLE', 'STAR_TABLE', 'GALAXY_TABLE', 'CLASS_SPEC', 'STAR_SPEC', 'GALAXY_SPEC']


### PRIMARY

The PRIMARY extension contains only a header with some basic information.

In [ ]:
hdus["PRIMARY"].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
L1_REF_0= 'stack_1002214.fit'  / L1 reference file                              
L1_REF_1= 'stack_1002213.fit'  / L1 reference file                              
L1_REF_2= '' / L1 reference file                                                
DATE-OBS= '20160908'           / L1: OBS-DATE                                   
OBSMODE = 'MOS     '           / L1: OBSMODE                                    
RES-OBS = 'LR      '           / L2: RES-DATE                                   
OBID    = '3191    '           / L1: OBID                                       
CHECKSUM= '96WLF4UL94ULC4UL'   / HDU checksum updated 2022-02-07T08:34:26       
DATASUM = '0       '        

In [ ]:
@via_netcdf
def read_primary_header(fn):
    hdr = fits.getheader(fn, "PRIMARY")
    return xr.Dataset(hdr)

In [ ]:
primary_header = read_primary_header(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [00:01<00:00, 14.72it/s]
Reading netCDF files... took 1.17 s.


In [ ]:
primary_header

<xarray.Dataset>
Dimensions:   (filename: 17)
Coordinates:
  * filename  (filename) object 'stack_1002214__stack_1002213_APS' ... 'stack...
Data variables: (12/13)
    L1_REF_0  (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    L1_REF_1  (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    CHECKSUM  (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    DATE-OBS  (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    OBID      (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    OBSMODE   (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    ...        ...
    L1_REF_2  (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    DATASUM   (filename) object dask.array<chunksize=(1,), meta=np.ndarray>
    SIMPLE    (filename) bool dask.array<chunksize=(1,), meta=np.ndarray>
    BITPIX    (filename) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    NAXIS     (filename) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    EXTEND    (filename) bool dask.array<chunksize=(1,), meta=np.ndarray>

### CLASS_TABLE

The CLASS_TABLE extension contains information from matching various templates to the spectra. The redshift and class of the best fitting template are given, as well as the full cross-correlation results of each template as a function of redshift.

In [ ]:
Table.read(hdus["CLASS_TABLE"])[:5]

APS_ID,TARGID,CNAME,Z,ZERR,ZWARN,CLASS,SUBCLASS,TARGSRVY,SNR,CHI2,DELTACHI2,NCOEFF,COEFF,NPIXELS,SRVY_CLASS,CZZ_GALAXY,CZZ_CHI2_GALAXY,CZZ_QSO,CZZ_CHI2_QSO,CZZ_STAR_A,CZZ_CHI2_STAR_A,CZZ_STAR_B,CZZ_CHI2_STAR_B,CZZ_STAR_CV,CZZ_CHI2_STAR_CV,CZZ_STAR_F,CZZ_CHI2_STAR_F,CZZ_STAR_G,CZZ_CHI2_STAR_G,CZZ_STAR_K,CZZ_CHI2_STAR_K,CZZ_STAR_M,CZZ_CHI2_STAR_M,CZZ_STAR_WD,CZZ_CHI2_STAR_WD
int64,str19,str20,float64,float64,int64,str6,str20,str5,float64,float64,float64,int64,float64[10],int64,str6,float64[1446],float64[1446],float64[1648],float64[1648],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101],float64[101]
6,2194729357709220864,WVE_20305297+6043446,-4.0019344842423317e-07,2.872323645641729e-07,0,STAR,G,GA-OC,80.8585205078125,554755.923579417,805070.5323945394,5,468100.5727658052 .. 0.0,24064,STAR,-0.0050000000000000044 .. 1.6997470838899442,1520002.1759944202 .. 1534760.8288547995,0.050000000000000044 .. 5.993414973633454,5506324.5438700905 .. 44146205.25364692,-0.002 .. 0.0020000000000000104,1416518.0181509315 .. 1421594.821586679,-0.002 .. 0.0020000000000000104,3228556.3115567537 .. 3421907.5609378116,-0.002 .. 0.0020000000000000104,2042201.5832182427 .. 2158862.1545822634,-0.002 .. 0.0020000000000000104,1348095.486672144 .. 1386622.2150982632,-0.002 .. 0.0020000000000000104,1393955.7263760436 .. 1407264.4820755827,-0.002 .. 0.0020000000000000104,1399040.3401225845 .. 1403099.6227823957,-0.002 .. 0.0020000000000000104,25837057.46569176 .. 26268618.42880452,-0.002 .. 0.0020000000000000104,1437025.3454249839 .. 1428496.1658297796
7,2194782031188201984,WVE_20293608+6053544,1.499601319652985e-06,2.532174880367198e-07,0,STAR,G,GA-OC,59.24796676635742,377247.2936920571,627222.5121733916,5,248626.97316464118 .. 0.0,24066,STAR,-0.0050000000000000044 .. 1.6997470838899442,1035448.6652695596 .. 1112362.6515732903,0.050000000000000044 .. 5.993414973633454,2437358.874697626 .. 33397709.59467234,-0.002 .. 0.0020000000000000104,1231590.5504169096 .. 1213826.499516298,-0.002 .. 0.0020000000000000104,3724171.0166908745 .. 3828501.8667422426,-0.002 .. 0.0020000000000000104,1419929.3911646432 .. 1454131.7132686356,-0.002 .. 0.0020000000000000104,1039482.7107725941 .. 1028249.8357967661,-0.002 .. 0.0020000000000000104,1077021.5935429083 .. 1074656.427525039,-0.002 .. 0.0020000000000000104,1084310.4672253616 .. 1076802.5728186287,-0.002 .. 0.0020000000000000104,9302997.710345287 .. 9497765.363115603,-0.002 .. 0.0020000000000000104,1439960.9039386632 .. 1392887.8330925538
9,nan,WVE_20302432+6109350,3.884123057975818,0.000315320763638615,0,QSO,--,--,-0.09991590678691864,19140.521881967783,1668.1164811488707,4,-0.00017569081326885457 .. 0.0,24064,QSO,-0.0050000000000000044 .. 1.6997470838899442,22155.90708714902 .. 22065.167842632192,0.050000000000000044 .. 5.993414973633454,22055.310292467315 .. 22476.36876375002,-0.002 .. 0.0020000000000000104,22214.470371253152 .. 22235.716655488533,-0.002 .. 0.0020000000000000104,22205.509968972696 .. 22221.96689152452,-0.002 .. 0.0020000000000000104,22165.69112405188 .. 22198.737900162287,-0.002 .. 0.0020000000000000104,22218.477644790917 .. 22226.900141815888,-0.002 .. 0.0020000000000000104,22242.95151823498 .. 22230.743632161368,-0.002 .. 0.0020000000000000104,21938.68102312989 .. 22024.8908973963,-0.002 .. 0.0020000000000000104,22074.559795463178 .. 22140.85548060489,-0.002 .. 0.0020000000000000104,22073.69681539014 .. 22126.65084839108
10,2194899953809075712,WVE_20295065+6121007,-1.8346905043006894e-07,1.4037287893115768e-07,0,STAR,G,GA-OC,191.4801483154297,6585662.659540394,2362631.262090465,5,3965728.799533826 .. 0.0,24065,STAR,-0.0050000000000000044 .. 1.6997470838899442,12065810.795406062 .. 11501262.51331262,0.050000000000000044 .. 5.993414973633454,40486449.14229764 .. 234462488.86398178,-0.002 .. 0.0020000000000000104,11231797.61808367 .. 10994823.34728493,-0.002 ..

In [ ]:
# |export


@via_netcdf
def read_class_table(fn):
    """Read the CLASS_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Chi-square values `CZZ_CHI2_*` for each template are further indexed by redshift `CZZ_*`.
    Coefficients `COEFF` and indexed by integers `I_COEFF`.
    """
    dat = Table.read(fn, "CLASS_TABLE")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    # convert CZZ columns to coordinates
    for c in list(cols):
        if c.startswith("CZZ") and "CHI2" not in c:
            czz_all = cols.pop(c)
            czz = czz_all.mean(axis=0).round(3)
            assert (czz == czz_all.round(3)).all()
            coords[c] = czz
    for c in cols:
        dims = ["APS_ID"]
        if c.startswith("CZZ"):
            dims += [c.replace("_CHI2", "")]
        elif c == "COEFF":
            dims += ["I_COEFF"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)

In [ ]:
class_table = read_class_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [00:08<00:00,  2.11it/s]
Reading netCDF files... took 3.15 s.


In [ ]:
class_table

<xarray.Dataset>
Dimensions:           (CZZ_QSO: 1648, CZZ_GALAXY: 1446, APS_ID: 1000,
                       CZZ_STAR_A: 101, CZZ_STAR_B: 101, CZZ_STAR_CV: 101,
                       CZZ_STAR_F: 101, CZZ_STAR_G: 101, CZZ_STAR_K: 101,
                       CZZ_STAR_M: 101, CZZ_STAR_WD: 101, filename: 17,
                       I_COEFF: 10)
Coordinates:
  * CZZ_QSO           (CZZ_QSO) float64 0.05 0.051 0.052 ... 5.977 5.985 5.993
  * CZZ_GALAXY        (CZZ_GALAXY) float64 -0.005 -0.004 -0.004 ... 1.698 1.7
  * APS_ID            (APS_ID) int32 1 2 3 4 5 6 ... 1003 1004 1005 1006 1007
  * CZZ_STAR_A        (CZZ_STAR_A) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_B        (CZZ_STAR_B) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_CV       (CZZ_STAR_CV) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_F        (CZZ_STAR_F) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_G        (CZZ_STAR_G) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_K        (CZZ_STAR_K) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_M        (CZZ_STAR_M) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * CZZ_STAR_WD       (CZZ_STAR_WD) float64 -0.002 -0.002 -0.002 ... 0.002 0.002
  * filename          (filename) object 'stack_1002214__stack_1002213_APS' .....
Dimensions without coordinates: I_COEFF
Data variables: (12/25)
    CZZ_CHI2_QSO      (filename, APS_ID, CZZ_QSO) float64 dask.array<chunksize=(1, 1000, 1648), meta=np.ndarray>
    CZZ_CHI2_GALAXY   (filename, APS_ID, CZZ_GALAXY) float64 dask.array<chunksize=(1, 1000, 1446), meta=np.ndarray>
    CZZ_CHI2_STAR_A   (filename, APS_ID, CZZ_STAR_A) float64 dask.array<chunksize=(1, 1000, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_B   (filename, APS_ID, CZZ_STAR_B) float64 dask.array<chunksize=(1, 1000, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_CV  (filename, APS_ID, CZZ_STAR_CV) float64 dask.array<chunksize=(1, 1000, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_F   (filename, APS_ID, CZZ_STAR_F) float64 dask.array<chunksize=(1, 1000, 101), meta=np.ndarray>
    ...                ...
    ZWARN             (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    SNR               (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    CHI2              (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    DELTACHI2         (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    NCOEFF            (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    NPIXELS           (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>

### STAR_TABLE

The STAR_TABLE extension contains measurements of stellar parameters.

In [ ]:
Table.read(hdus["STAR_TABLE"])[:5]

APS_ID,TARGID,CNAME,VRAD,VRAD_ERR,SKEWNESS_RVS,KURTOSIS_RVS,LOGG_RVS,TEFF_RVS,VSINI_RVS,FEH_RVS,ALPHA_RVS,LOGG_ERR_RVS,TEFF_ERR_RVS,FEH_ERR_RVS,ALPHA_ERR_RVS,SNR_RVS,CHISQ_TOT_RVS,TEFF,TEFF_ERR,LOGG,LOGG_ERR,FEH,FEH_ERR,ALPHA,ALPHA_ERR,MICRO,MICRO_ERR,COVAR,ELEM,ELEM_ERR,SNR_FR,CHISQ_TOT,FLAG_FR
,,,km / s,km / s,km / s,km / s,,K,km / s,,,,K,,,,,K,K,,,,,,,,,,,,,,
int64,str19,str20,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,"float64[5,5]",float64[2],float64[2],float64,float64,int64
6,2194729357709220864,WVE_20305297+6043446,-0.504667279357346,0.08037169618493809,-1.1431464622005103e-05,3.19239481150184,4.684689599197517,6499.997214402617,0.1,-0.20586231627459092,0.13279489609969292,0.0005289232639313837,2.3204548308251103,0.0033694455573900525,0.00016811932875870653,79.89289,397126.31036438484,6584.2,8.8446,4.6682,0.015582,-0.036924,0.014104,0.0047364,0.0090408,0.11193,0.0085853,0.00019893 .. 0.0002428,-- .. --,-- .. --,81.673,10.327614057613975,0
7,2194782031188201984,WVE_20293608+6053544,0.2559520433300336,0.07042579140039812,-0.00030917720175791473,3.644997862586809,4.8879438346097555,5607.27268137104,0.11002704081784744,-0.0883044419236057,0.07064344475367432,--,--,--,0.0002226714652847583,58.40185,280595.78067210823,5660.1,4.3074,4.8529,0.012911,0.08011,0.010314,-0.0099316,0.0074031,0.11948,0.0080503,0.00010637 .. 0.00016668,-- .. --,-- .. --,60.632,7.824561348744246,0
10,2194899953809075712,WVE_20295065+6121007,-0.23838661024915722,0.01074607009882184,0.0025041900902526117,141.8798724733982,4.094096425894923,6642.273031802337,0.12677331868578,-0.2899471799447083,0.23195817045466272,0.002536254494286244,1.176158038082157,0.0012506337539533012,7.143250249368568e-05,189.34898,2995109.3328878987,6683.2,4.7714,4.0131,0.0073783,-0.13907,0.0032635,-0.022532,0.002977,0.080661,0.0034098,1.0651e-05 .. 5.444e-05,-- .. --,-- .. --,192.37,123.11189039637796,0
14,2194898476340333312,WVE_20302124+6115371,-0.038190230133544924,0.017684394565806658,-0.012466189073841406,52.389254289435115,4.836567397625863,6088.082643995249,0.1,-0.1359347051674617,0.07812281865425875,--,--,--,0.00010901310222992684,122.26987,1300946.1092753701,6168.8,4.2236,4.8153,0.007387,0.031704,0.0074623,-0.029435,0.0048437,0.13681,0.0047332,5.5686e-05 .. 5.4567e-05,-- .. --,-- .. --,126.23,31.608217127583483,0
15,2194898029663655552,WVE_20300721+6113192,0.43431960412584353,0.06046423522043829,-0.0004852317881561887,4.632701166504769,5.548483038295787,5426.016738514633,0.10560996900239475,-0.02401220666885248,0.019210406880426308,0.0006487120608506682,3.4759478866753297,0.004243880585802961,0.0002315827517515149,55.351944,278617.1935241955,5185.9,4.0236,5.0,0.012225,0.0010018,0.0096827,0.047342,0.0049487,0.20248,0.0069128,9.3754e-05 .. 0.00014944,-- .. --,-- .. --,57.739,7.182076058301705,0


In [ ]:
dat = Table.read(fn, "STAR_TABLE")
cols = dict(dat)

In [ ]:
# |export


@via_netcdf
def read_star_table(fn):
    """Read the STAR_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    The covariance matrix `COVAR` is additionally indexed by `I_COVAR`, `J_COVAR`.
    The elements `ELEM` and `ELEM_ERR` are additionally indexed by `I_ELEM`.
    """
    dat = Table.read(fn, "STAR_TABLE")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    coords["I_COVAR"] = coords["J_COVAR"] = ["TEFF", "LOGG", "FEH", "ALPHA", "MICRO"]
    for c in cols:
        dims = ["APS_ID"]
        if c == "COVAR":
            dims += ["I_COVAR", "J_COVAR"]
        elif "ELEM" in c:
            dims += ["I_ELEM"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)

In [ ]:
star_table = read_star_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 18.90it/s]
Reading netCDF files... took 3.18 s.


In [ ]:
star_table

<xarray.Dataset>
Dimensions:        (APS_ID: 1000, I_COVAR: 5, J_COVAR: 5, filename: 17,
                    I_ELEM: 2)
Coordinates:
  * APS_ID         (APS_ID) int32 1 2 3 4 5 6 ... 1002 1003 1004 1005 1006 1007
  * I_COVAR        (I_COVAR) object 'TEFF' 'LOGG' 'FEH' 'ALPHA' 'MICRO'
  * J_COVAR        (J_COVAR) object 'TEFF' 'LOGG' 'FEH' 'ALPHA' 'MICRO'
  * filename       (filename) object 'stack_1002214__stack_1002213_APS' ... '...
Dimensions without coordinates: I_ELEM
Data variables: (12/33)
    CNAME          (filename, APS_ID) object dask.array<chunksize=(1, 1000), meta=np.ndarray>
    TARGID         (filename, APS_ID) object dask.array<chunksize=(1, 1000), meta=np.ndarray>
    COVAR          (filename, APS_ID, I_COVAR, J_COVAR) float64 dask.array<chunksize=(1, 1000, 5, 5), meta=np.ndarray>
    ELEM           (filename, APS_ID, I_ELEM) float64 dask.array<chunksize=(1, 1000, 2), meta=np.ndarray>
    ELEM_ERR       (filename, APS_ID, I_ELEM) float64 dask.array<chunksize=(1, 1000, 2), meta=np.ndarray>
    VRAD           (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    ...             ...
    ALPHA_ERR      (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    MICRO          (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    MICRO_ERR      (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    SNR_FR         (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    CHISQ_TOT      (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    FLAG_FR        (filename, APS_ID) float64 dask.array<chunksize=(1, 1000), meta=np.ndarray>

### GALAXY_TABLE

The GALAXY_TABLE extension contains measurements of galaxy parameters, including Hubble-flow corrected redshifts, stellar kinematics, line fits and indices.

In [ ]:
Table.read(hdus["GALAXY_TABLE"], unit_parse_strict="silent")[:5]

APS_ID,TARGID,CNAME,ZCORR,V,SIGMA,H3,H4,H5,H6,ERR_ZCORR,ERR_V,ERR_SIGMA,ERR_H3,ERR_H4,ERR_H5,ERR_H6,FORM_ERR_ZCORR,FORM_ERR_V,FORM_ERR_SIGMA,FORM_ERR_H3,FORM_ERR_H4,FORM_ERR_H5,FORM_ERR_H6,FLUX_HeII_3203.15,AMPL_HeII_3203.15,Z_HeII_3203.15,SIGMA_HeII_3203.15,AON_HeII_3203.15,EBMV_HeII_3203.15,ERR_FLUX_HeII_3203.15,ERR_AMPL_HeII_3203.15,ERR_Z_HeII_3203.15,ERR_SIGMA_HeII_3203.15,ERR_EBMV_HeII_3203.15,FLUX_[NeV]_3345.81,AMPL_[NeV]_3345.81,Z_[NeV]_3345.81,SIGMA_[NeV]_3345.81,AON_[NeV]_3345.81,EBMV_[NeV]_3345.81,ERR_FLUX_[NeV]_3345.81,ERR_AMPL_[NeV]_3345.81,ERR_Z_[NeV]_3345.81,ERR_SIGMA_[NeV]_3345.81,ERR_EBMV_[NeV]_3345.81,FLUX_[NeV]_3425.81,AMPL_[NeV]_3425.81,Z_[NeV]_3425.81,SIGMA_[NeV]_3425.81,AON_[NeV]_3425.81,EBMV_[NeV]_3425.81,ERR_FLUX_[NeV]_3425.81,ERR_AMPL_[NeV]_3425.81,ERR_Z_[NeV]_3425.81,ERR_SIGMA_[NeV]_3425.81,ERR_EBMV_[NeV]_3425.81,FLUX_[OII]_3726.03,AMPL_[OII]_3726.03,Z_[OII]_3726.03,SIGMA_[OII]_3726.03,AON_[OII]_3726.03,EBMV_[OII]_3726.03,ERR_FLUX_[OII]_3726.03,ERR_AMPL_[OII]_3726.03,ERR_Z_[OII]_3726.03,ERR_SIGMA_[OII]_3726.03,ERR_EBMV_[OII]_3726.03,FLUX_[OII]_3728.73,AMPL_[OII]_3728.73,Z_[OII]_3728.73,SIGMA_[OII]_3728.73,AON_[OII]_3728.73,EBMV_[OII]_3728.73,ERR_FLUX_[OII]_3728.73,ERR_AMPL_[OII]_3728.73,ERR_Z_[OII]_3728.73,ERR_SIGMA_[OII]_3728.73,ERR_EBMV_[OII]_3728.73,FLUX_[NeIII]_3868.69,AMPL_[NeIII]_3868.69,Z_[NeIII]_3868.69,SIGMA_[NeIII]_3868.69,AON_[NeIII]_3868.69,EBMV_[NeIII]_3868.69,ERR_FLUX_[NeIII]_3868.69,ERR_AMPL_[NeIII]_3868.69,ERR_Z_[NeIII]_3868.69,ERR_SIGMA_[NeIII]_3868.69,ERR_EBMV_[NeIII]_3868.69,FLUX_[NeIII]_3967.40,AMPL_[NeIII]_3967.40,Z_[NeIII]_3967.40,SIGMA_[NeIII]_3967.40,AON_[NeIII]_3967.40,EBMV_[NeIII]_3967.40,ERR_FLUX_[NeIII]_3967.40,ERR_AMPL_[NeIII]_3967.40,ERR_Z_[NeIII]_3967.40,ERR_SIGMA_[NeIII]_3967.40,ERR_EBMV_[NeIII]_3967.40,FLUX_[OIII]_4363.15,AMPL_[OIII]_4363.15,Z_[OIII]_4363.15,SIGMA_[OIII]_4363.15,AON_[OIII]_4363.15,EBMV_[OIII]_4363.15,ERR_FLUX_[OIII]_4363.15,ERR_AMPL_[OIII]_4363.15,ERR_Z_[OIII]_4363.15,ERR_SIGMA_[OIII]_4363.15,ERR_EBMV_[OIII]_4363.15,FLUX_HeII_4685.74,AMPL_HeII_4685.74,Z_HeII_4685.74,SIGMA_HeII_4685.74,AON_HeII_4685.74,EBMV_HeII_4685.74,ERR_FLUX_HeII_4685.74,ERR_AMPL_HeII_4685.74,ERR_Z_HeII_4685.74,ERR_SIGMA_HeII_4685.74,ERR_EBMV_HeII_4685.74,FLUX_[ArIV]_4711.30,AMPL_[ArIV]_4711.30,Z_[ArIV]_4711.30,SIGMA_[ArIV]_4711.30,AON_[ArIV]_4711.30,EBMV_[ArIV]_4711.30,ERR_FLUX_[ArIV]_4711.30,ERR_AMPL_[ArIV]_4711.30,ERR_Z_[ArIV]_4711.30,ERR_SIGMA_[ArIV]_4711.30,ERR_EBMV_[ArIV]_4711.30,FLUX_[ArIV]_4740.10,AMPL_[ArIV]_4740.10,Z_[ArIV]_4740.10,SIGMA_[ArIV]_4740.10,AON_[ArIV]_4740.10,EBMV_[ArIV]_4740.10,ERR_FLUX_[ArIV]_4740.10,ERR_AMPL_[ArIV]_4740.10,ERR_Z_[ArIV]_4740.10,ERR_SIGMA_[ArIV]_4740.10,ERR_EBMV_[ArIV]_4740.10,FLUX_Hb_4861.32,AMPL_Hb_4861.32,Z_Hb_4861.32,SIGMA_Hb_4861.32,AON_Hb_4861.32,EBMV_Hb_4861.32,ERR_FLUX_Hb_4861.32,ERR_AMPL_Hb_4861.32,ERR_Z_Hb_4861.32,ERR_SIGMA_Hb_4861.32,ERR_EBMV_Hb_4861.32,FLUX_[OIII]_5006.77,AMPL_[OIII]_5006.77,Z_[OIII]_5006.77,SIGMA_[OIII]_5006.77,AON_[OIII]_5006.77,EBMV_[OIII]_5006.77,ERR_FLUX_[OIII]_5006.77,ERR_AMPL_[OIII]_5006.77,ERR_Z_[OIII]_5006.77,ERR_SIGMA_[OIII]_5006.77,ERR_EBMV_[OIII]_5006.77,FLUX_[NI]_5197.90,AMPL_[NI]_5197.90,Z_[NI]_5197.90,SIGMA_[NI]_5197.90,AON_[NI]_5197.90,EBMV_[NI]_5197.90,ERR_FLUX_[NI]_5197.90,ERR_AMPL_[NI]_5197.90,ERR_Z_[NI]_5197.90,ERR_SIGMA_[NI]_5197.90,ERR_EBMV_[NI]_5197.90,FLUX_[NI]_5200.39,AMPL_[NI]_5200.39,Z_[NI]_5200.39,SIGMA_[NI]_5200.39,AON_[NI]_5200.39,EBMV_[NI]_5200.39,ERR_FLUX_[NI]_5200.39,ERR_AMPL_[NI]_5200.39,ERR_Z_[NI]_5200.39,ERR_SIGMA_[NI]_5200.39,ERR_EBMV_[NI]_5200.39,FLUX_HeI_5875.60,AMPL_HeI_5875.60,Z_HeI_5875.60,SIGMA_HeI_5875.60,AON_HeI_5875.60,EBMV_HeI_5875.60,ERR_FLUX_HeI_5875.60,ERR_AMPL_HeI_5875.60,ERR_Z_HeI_5875.60,ERR_SIGMA_HeI_5875.60,ERR_EBMV_HeI_5875.60,FLUX_[OI]_6300.20,AMPL_[OI]_6300.20,Z_[OI]_6300.20,SIGMA_[OI]_6300.20,AON_[OI]_6300.20,EBMV_[OI]_6300.20,ERR_FLUX_[OI]_6300.20,ERR_AMPL_[OI]_6300.20,ERR_Z_[OI]_6300.20,ERR_SIGMA_[OI]_6300.20,ERR_EBMV_[OI]_6300.20,FLUX_Ha_6562.80,AMPL_Ha_6

In [ ]:
# |export


def _not_line_col(c):
    """Identify columns that do not contain line measurements."""
    c = c.replace("ERR_", "")
    for n in ["EBMV0", "EBMV1", "FLUX", "AMPL", "Z", "SIGMA", "AON", "FWHM"]:
        if c.startswith(n + "_"):
            return False
    return True


def _process_line_quantities(cols, lines):
    """Process line quantities.

    Quantities with multiple elements are split into separate columns.

    The supplied `cols` dictionary is modified in-place.
    """
    line_quantities = []
    for c in list(cols):
        if c.endswith(lines[0]):
            qty = c.replace("_" + lines[0], "")
            ndim = cols[c].ndim
            if ndim == 1:
                line_quantities.append(qty)
            elif ndim == 2:
                nq = cols[c].shape[1]
                for i in range(nq):
                    line_quantities.append(f"{qty}{i}")
                for line in lines:
                    oldcol = cols.pop(f"{qty}_{line}")
                    for i in range(nq):
                        cols[f"{qty}{i}_{line}"] = oldcol[:, i]
    return line_quantities


@via_netcdf
def read_galaxy_table(fn):
    """Read the GALAXY_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    The line measurements are additionally indexed by the measurement quantity `QTY`
    and the line name `LINE`.
    The index measurements are additionally indexed by the index name `INDEX`.
    """
    # TODO: add units where missing
    dat = Table.read(fn, "GALAXY_TABLE", unit_parse_strict="silent")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    coords["LINE"] = [c.replace("FLUX_", "") for c in cols if c.startswith("FLUX")]
    coords["QTY"] = _process_line_quantities(cols, coords["LINE"])
    coords["INDEX"] = [c for i, c in enumerate(cols) if (i > 100 and _not_line_col(c))]
    line_cols = [
        [cols.pop(f"{qty}_{line}") for line in coords["LINE"]] for qty in coords["QTY"]
    ]
    index_cols = [cols.pop(idx) for idx in coords["INDEX"]]
    out_cols = {}
    for i, c in enumerate(cols):
        dims = ["APS_ID"]
        out_cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    out_cols["LINES"] = xr.Variable(["QTY", "LINE", "APS_ID"], line_cols)
    out_cols["INDICES"] = xr.Variable(["INDEX", "APS_ID"], index_cols)
    return xr.Dataset(out_cols, coords)

In [ ]:
galaxy_table = read_galaxy_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 6911.90it/s]
Reading netCDF files... took 2.76 s.


In [ ]:
galaxy_table

<xarray.Dataset>
Dimensions:         (INDEX: 292, APS_ID: 999, LINE: 22, QTY: 13, filename: 17)
Coordinates:
  * INDEX           (INDEX) object 'BL1719' 'ERR_BL1719' ... 'ERR_MgI2.28'
  * APS_ID          (APS_ID) int32 1 2 3 4 5 6 ... 1002 1003 1004 1005 1006 1007
  * LINE            (LINE) object 'HeII_3203.15' ... '[ArIII]_7135.67'
  * QTY             (QTY) object 'FLUX' 'AMPL' 'Z' ... 'ERR_EBMV0' 'ERR_EBMV1'
  * filename        (filename) object 'stack_1002214__stack_1002213_APS' ... ...
Data variables: (12/26)
    INDICES         (filename, INDEX, APS_ID) float64 dask.array<chunksize=(1, 292, 999), meta=np.ndarray>
    TARGID          (filename, APS_ID) object dask.array<chunksize=(1, 999), meta=np.ndarray>
    CNAME           (filename, APS_ID) object dask.array<chunksize=(1, 999), meta=np.ndarray>
    ZCORR           (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    V               (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    SIGMA           (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    ...              ...
    FORM_ERR_H3     (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    FORM_ERR_H4     (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    FORM_ERR_H5     (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    FORM_ERR_H6     (filename, APS_ID) float64 dask.array<chunksize=(1, 999), meta=np.ndarray>
    FWHM_FLAG       (filename, APS_ID) float32 dask.array<chunksize=(1, 999), meta=np.ndarray>
    LINES           (filename, QTY, LINE, APS_ID) float64 dask.array<chunksize=(1, 13, 22, 999), meta=np.ndarray>

### CLASS_SPEC

The CLASS_SPEC extension contains spectra and model fits, performed separately for the red and blue arms.

In [ ]:
Table.read(hdus["CLASS_SPEC"], unit_parse_strict="silent")[:5]

APS_ID,TARGID,CNAME,LAMBDA_RR_B,FLUX_RR_B,IVAR_RR_B,MODEL_RR_B,LAMBDA_RR_R,FLUX_RR_R,IVAR_RR_R,MODEL_RR_R
,,,Angstrom,1e-18 erg / (Angstrom s cm2),1e+36 cm4 Angstrom2 / (erg2 s2),1e-18 erg / (Angstrom s cm2),Angstrom,1e-18 erg / (Angstrom s cm2),1e+36 cm4 Angstrom2 / (erg2 s2),1e-18 erg / (Angstrom s cm2)
int64,str19,str20,float64[9648],float32[9648],float32[9648],float64[9648],float64[15288],float32[15288],float32[15288],float64[15288]
6,2194729357709220864,WVE_20305297+6043446,3677.046767844175 .. 6089.435411423556,-- .. --,0.0 .. 0.0,-- .. --,5773.600816055148 .. 9596.381567404502,-- .. --,0.0 .. 0.0,-- .. --
7,2194782031188201984,WVE_20293608+6053544,3677.046767844175 .. 6089.435411423556,-- .. --,0.0 .. 0.0,-- .. --,5773.600816055148 .. 9596.381567404502,-- .. --,0.0 .. 0.0,-- .. --
9,nan,WVE_20302432+6109350,3677.046767844175 .. 6089.435411423556,-- .. --,0.0 .. 0.0,-- .. --,5773.600816055148 .. 9596.381567404502,-- .. --,0.0 .. 0.0,-- .. --
10,2194899953809075712,WVE_20295065+6121007,3677.046767844175 .. 6089.435411423556,-- .. --,0.0 .. 0.0,-- .. --,5773.600816055148 .. 9596.381567404502,-- .. --,0.0 .. 0.0,-- .. --
11,nan,WVE_20304900+6059357,3677.046767844175 .. 6089.435411423556,-- .. --,0.0 .. 0.0,-- .. --,5773.600816055148 .. 9596.381567404502,-- .. --,0.0 .. 0.0,-- .. --


In [ ]:
# |export


@via_netcdf
def read_class_spec(fn):
    """Read the CLASS_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by wavelength `LAMBDA_{B,R}`.
    """
    dat = Table.read(fn, "CLASS_SPEC")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in list(cols):
        if c.startswith("LAMBDA"):
            band = c.split("_")[-1]
            wl_all = cols.pop(c)
            wl = wl_all.mean(axis=0).round(3)
            assert (wl == wl_all.round(3)).all()
            coords[f"LAMBDA_{band}"] = wl
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_B"):
            dims += ["LAMBDA_B"]
        elif c.endswith("_R"):
            dims += ["LAMBDA_R"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)

In [ ]:
class_spec = read_class_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [01:00<00:00,  3.57s/it]
Reading netCDF files... took 1.65 s.


In [ ]:
class_spec

<xarray.Dataset>
Dimensions:     (LAMBDA_R: 15288, LAMBDA_B: 9648, APS_ID: 1000, filename: 17)
Coordinates:
  * LAMBDA_R    (LAMBDA_R) float64 5.774e+03 5.774e+03 ... 9.596e+03 9.596e+03
  * LAMBDA_B    (LAMBDA_B) float64 3.677e+03 3.677e+03 ... 6.089e+03 6.089e+03
  * APS_ID      (APS_ID) int32 1 2 3 4 5 6 7 ... 1002 1003 1004 1005 1006 1007
  * filename    (filename) object 'stack_1002214__stack_1002213_APS' ... 'sta...
Data variables:
    FLUX_RR_R   (filename, APS_ID, LAMBDA_R) float32 dask.array<chunksize=(1, 1000, 15288), meta=np.ndarray>
    IVAR_RR_R   (filename, APS_ID, LAMBDA_R) float32 dask.array<chunksize=(1, 1000, 15288), meta=np.ndarray>
    MODEL_RR_R  (filename, APS_ID, LAMBDA_R) float64 dask.array<chunksize=(1, 1000, 15288), meta=np.ndarray>
    FLUX_RR_B   (filename, APS_ID, LAMBDA_B) float32 dask.array<chunksize=(1, 1000, 9648), meta=np.ndarray>
    IVAR_RR_B   (filename, APS_ID, LAMBDA_B) float32 dask.array<chunksize=(1, 1000, 9648), meta=np.ndarray>
    MODEL_RR_B  (filename, APS_ID, LAMBDA_B) float64 dask.array<chunksize=(1, 1000, 9648), meta=np.ndarray>
    CNAME       (filename, APS_ID) object dask.array<chunksize=(1, 1000), meta=np.ndarray>
    TARGID      (filename, APS_ID) object dask.array<chunksize=(1, 1000), meta=np.ndarray>

### STAR_SPEC

The STAR_SPEC extension contains spectra and model fits for stellar measurements.

In [ ]:
Table.read(hdus["STAR_SPEC"], unit_parse_strict="silent")[:5]

APS_ID,TARGID,CNAME,LAMBDA_RVS_B,FLUX_RVS_B,ERROR_RVS_B,MODEL_RVS_B,LAMBDA_RVS_R,FLUX_RVS_R,ERROR_RVS_R,MODEL_RVS_R,LAMBDA_FR_C,FLUX_FR_C,ERROR_FR_C,MODEL_FR_C
,,,Angstrom,1e-18 erg / (Angstrom s cm2),1e+36 cm4 Angstrom2 / (erg2 s2),1e-18 erg / (Angstrom s cm2),Angstrom,1e-18 erg / (Angstrom s cm2),1e+36 cm4 Angstrom2 / (erg2 s2),1e-18 erg / (Angstrom s cm2),Angstrom,1e-18 erg / (Angstrom s cm2),1e-18 erg / (Angstrom s cm2),1e-18 erg / (Angstrom s cm2)
int64,str19,str20,float64[9648],float32[9648],float32[9648],float64[9648],float64[15288],float32[15288],float32[15288],float64[15288],float64[23672],float64[23672],float64[23672],float64[23672]
6,2194729357709220864,WVE_20305297+6043446,3676.0 .. 6087.75,2001.6849 .. --,213.63112 .. --,1892.7123566768062 .. --,5772.0 .. 9593.75,-- .. --,-- .. --,-- .. --,3676.006 .. 9593.766,1.04548 .. --,213.6311 .. --,1.08691 .. --
7,2194782031188201984,WVE_20293608+6053544,3676.0 .. 6087.75,1092.8457 .. --,184.28183 .. --,978.2953653033654 .. --,5772.0 .. 9593.75,-- .. --,-- .. --,-- .. --,3675.997 .. 9593.742,1.18096 .. --,184.2818 .. --,1.19583 .. --
10,2194899953809075712,WVE_20295065+6121007,3676.0 .. 6087.75,15929.538 .. --,693.98785 .. --,13576.86173032569 .. --,5772.0 .. 9593.75,-- .. --,-- .. --,-- .. --,3676.003 .. 9593.758,1.04994 .. --,693.9878 .. --,1.05766 .. --
14,2194898476340333312,WVE_20302124+6115371,3676.0 .. 6087.75,4002.0413 .. --,281.22397 .. --,4330.706871786213 .. --,5772.0 .. 9593.75,-- .. --,-- .. --,-- .. --,3676.0 .. 9593.751,1.0516 .. --,281.224 .. --,1.12999 .. --
15,2194898029663655552,WVE_20300721+6113192,3676.0 .. 6087.75,616.2874 .. --,141.23196 .. --,605.4935624612925 .. --,5772.0 .. 9593.75,-- .. --,-- .. --,-- .. --,3675.995 .. 9593.736,1.27347 .. --,141.232 .. --,1.28379 .. --


In [ ]:
# |export


@via_netcdf
def read_star_spec(fn):
    """Read the STAR_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by wavelength bin `LAMBIN_{R,B,C}`,
    which does *not* correspond to the same wavelength for each spectrum.
    """
    dat = Table.read(fn, "STAR_SPEC")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_B"):
            dims += ["LAMBIN_B"]
        elif c.endswith("_R"):
            dims += ["LAMBIN_R"]
        elif c.endswith("_C"):
            dims += ["LAMBIN_C"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)

In [ ]:
star_spec = read_star_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [01:28<00:00,  5.23s/it]
Reading netCDF files... took 2.31 s.


### GALAXY_SPEC

The GALAXY_SPEC extension contains log-wavelength-binned spectra and model fits by PPXF and GANDALF.

In [ ]:
Table.read(hdus["GALAXY_SPEC"], unit_parse_strict="silent")[:5]

APS_ID,TARGID,CNAME,LOGLAM_PPXF,FLUX_PPXF,ERROR_PPXF,MODEL_PPXF,GOODPIX_PPXF,LOGLAM_GAND,FLUX_GAND,ERROR_GAND,MODEL_GAND,EMISSION_GAND,FLUX_CLEAN_GAND,MODEL_CLEAN_GAND,GOODPIX_GAND
,,,Angstrom,1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),,Angstrom,1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),1.000000e-18 erg/(s cm**2 Angstrom),
int32,str40,str40,float64[23671],float64[23671],float64[23671],float64[23671],int32[23671],float64[23671],float64[23671],float64[23671],float64[23671],float64[23671],float64[23671],float64[23671],int32[23671]
9,nan,WVE_20302432+6109350,6.6235769895145955 .. 7.58282961178434,6.440318584442138 .. 0.0,152.73657472242692 .. 8.01897405078567e+17,-- .. --,-1 .. -1,6.6235769895145955 .. 7.58282961178434,6.440318584442138 .. 0.0,152.73657472242692 .. 8.01897405078567e+17,-- .. --,-- .. --,-- .. --,-- .. --,-1 .. -1
11,nan,WVE_20304900+6059357,6.643871176172771 .. 7.603123798442517,-0.6244945526123047 .. 0.0,142.83660368475964 .. 7.938016210709007e+17,-- .. --,-1 .. -1,6.643871176172771 .. 7.603123798442517,-0.6244945526123047 .. 0.0,142.83660368475964 .. 7.938016210709007e+17,-- .. --,-- .. --,-- .. --,-- .. --,-1 .. -1
16,2194929056507729024,WVE_20300552+6131530,7.265446142144053 .. 8.2246987644138,21815.48828125 .. 0.0,812.7357864989899 .. 5.817530817381764e+17,-- .. --,18313 .. -1,7.265446142144053 .. 8.2246987644138,21815.48828125 .. 0.0,812.7357864989899 .. 5.817530817381764e+17,-- .. --,-- .. --,-- .. --,-- .. --,18283 .. -1
24,2194885282200728192,WVE_20305415+6110026,7.263850041757662 .. 8.22310266402741,52495.54296875 .. 0.0,1549.7973241703885 .. 5.822175352013871e+17,-- .. --,18322 .. -1,7.263850041757662 .. 8.22310266402741,52495.54296875 .. 0.0,1549.7973241703885 .. 5.822175352013871e+17,-- .. --,-- .. --,-- .. --,-- .. --,18322 .. -1
28,2194925586174161408,WVE_20304183+6128360,8.209544222345832 .. 9.168796844615581,18535.904296875 .. 0.0,437.82596418090924 .. 3.628527336139256e+17,20082.020095378408 .. 12294.895217924814,0 .. -1,8.209544222345832 .. 9.168796844615581,18535.904296875 .. 0.0,437.82596418090924 .. 3.628527336139256e+17,18963.632075353125 .. 6566.8276065747905,3.189522259389025e-51 .. 0.0,18535.904296875 .. 0.0,18963.632075353125 .. 6566.8276065747905,0 .. -1


In [ ]:
# |export


@via_netcdf
def read_galaxy_spec(fn):
    """Read the GALAXY_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by log-wavelength bin `LOGLAMBIN`,
    which does *not* correspond to the same wavelength for each spectrum.
    """
    dat = Table.read(fn, "GALAXY_SPEC", unit_parse_strict="silent")
    cols = dict(dat)
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_PPXF") or c.endswith("_GAND"):
            dims += ["LOGLAMBIN"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)

In [ ]:
galaxy_spec = read_galaxy_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 17/17 [02:56<00:00, 10.37s/it]
Reading netCDF files... took 2.79 s.


In [ ]:
galaxy_spec

<xarray.Dataset>
Dimensions:           (APS_ID: 999, filename: 17, LOGLAMBIN: 23671)
Coordinates:
  * APS_ID            (APS_ID) int32 1 2 3 4 5 6 ... 1003 1004 1005 1006 1007
  * filename          (filename) object 'stack_1002214__stack_1002213_APS' .....
Dimensions without coordinates: LOGLAMBIN
Data variables: (12/15)
    LOGLAM_PPXF       (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    FLUX_PPXF         (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    ERROR_PPXF        (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    MODEL_PPXF        (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    GOODPIX_PPXF      (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    LOGLAM_GAND       (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    ...                ...
    EMISSION_GAND     (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    FLUX_CLEAN_GAND   (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    MODEL_CLEAN_GAND  (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    GOODPIX_GAND      (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 999, 23671), meta=np.ndarray>
    TARGID            (filename, APS_ID) object dask.array<chunksize=(1, 999), meta=np.ndarray>
    CNAME             (filename, APS_ID) object dask.array<chunksize=(1, 999), meta=np.ndarray>

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()